In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import pickle
import pathlib
import os
import copy
import datetime
import time
# sklearn?


In [2]:
import missingno as msno

In [3]:
data_path = "Data/"
data = pd.read_csv(data_path+"subset-3-sous-ensemble-3.csv", encoding='latin1')
data.head()

,LEVEL1ID,LEVEL2ID,LEVEL3ID,LEVEL4ID,LEVEL5ID,SURVEYR,BYCOND,DESCRIP_E,DESCRIP_F,DEMCODE,...,SCORE100,ANSCOUNT,DEPT_E,DEPT_F,INDICATORID,INDICATORENG,INDICATORFRA,SUBINDICATORID,SUBINDICATORENG,SUBINDICATORFRA
0,0,0,0,0,0,2020,Q115 = 1,Male gender,Genre masculin,2011,...,75,73497,Public Service,Fonction publique,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
1,1,0,0,0,0,2020,Q115 = 1,Male gender,Genre masculin,2011,...,79,11550,Canada Revenue Agency,Agence du revenu du Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
2,2,0,0,0,0,2020,Q115 = 1,Male gender,Genre masculin,2011,...,80,5626,Employment and Social Development Canada,Emploi et Développement social Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
3,3,0,0,0,0,2020,Q115 = 1,Male gender,Genre masculin,2011,...,72,7308,Department of National Defence,Ministère de la Défense nationale,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement
4,4,0,0,0,0,2020,Q115 = 1,Male gender,Genre masculin,2011,...,64,3252,Correctional Service Canada,Service correctionnel Canada,4,WORKPLACE,MILIEU DE TRAVAIL,14,Physical environment and equipment,Environnement physique et équipement


In [4]:
print(data.shape)
print(data.columns)

(1405579, 35)
Index(['LEVEL1ID', 'LEVEL2ID', 'LEVEL3ID', 'LEVEL4ID', 'LEVEL5ID', 'SURVEYR',
       'BYCOND', 'DESCRIP_E', 'DESCRIP_F', 'DEMCODE', 'QUESTION', 'TITLE_E',
       'TITLE_F', 'ANSWER1', 'ANSWER2', 'ANSWER3', 'ANSWER4', 'ANSWER5',
       'ANSWER6', 'ANSWER7', 'MOST_POSITIVE_OR_LEAST_NEGATIVE',
       'NEUTRAL_OR_MIDDLE_CATEGORY', 'MOST_NEGATIVE_OR_LEAST_POSITIVE',
       'AGREE', 'SCORE5', 'SCORE100', 'ANSCOUNT', 'DEPT_E', 'DEPT_F',
       'INDICATORID', 'INDICATORENG', 'INDICATORFRA', 'SUBINDICATORID',
       'SUBINDICATORENG', 'SUBINDICATORFRA'],
      dtype='object')


In [5]:
empty = []
for col in data.columns:
    uni = data[col].unique()
    if len(uni)<2:
        print(col, uni)
        data.drop(columns=[col], inplace=True)
print(data.shape)

LEVEL2ID [0]
LEVEL3ID [0]
LEVEL4ID [0]
LEVEL5ID [0]
(1405579, 31)


In [6]:
data.describe()

,LEVEL1ID,SURVEYR,DEMCODE,INDICATORID,SUBINDICATORID
count,1.405579e+06,1.405579e+06,1.405579e+06,1.405579e+06,1.405579e+06
mean,4.118951e+01,2.019480e+03,2.041390e+03,3.933464e+00,1.456888e+01
std,3.007655e+01,6.753379e-01,1.858147e+01,8.859693e-01,5.516655e+00
min,0.000000e+00,2.018000e+03,2.011000e+03,1.000000e+00,1.000000e+00
25%,1.500000e+01,2.019000e+03,2.023000e+03,4.000000e+00,1.100000e+01
50%,3.400000e+01,2.020000e+03,2.044000e+03,4.000000e+00,1.600000e+01
75%,7.000000e+01,2.020000e+03,2.058000e+03,4.000000e+00,1.700000e+01
max,9.500000e+01,2.020000e+03,2.069000e+03,6.000000e+00,2.300000e+01


In [8]:
data['SUBINDICATORFRA'].unique()

array(['Environnement physique et équipement',
       'Utilisation des langues officielles',
       "Compatibilité entre la personne et l'emploi et perfectionnement",
       'Équilibre travail-vie privée et charge de travail',
       'Mobilisation des employés', 'Gestion du rendement',
       'Buts organisationnels', 'Habilitation',
       'Rendement organisationnel', 'Milieu de travail éthique',
       'Diversité et inclusion', 'Anti-racisme', 'Superviseur immédiat',
       'Haute direction', 'Mobilité et maintien en poste', 'Harcèlement',
       'Discrimination', 'Stress lié au travail',
       'Milieu de travail sain sur le plan psychologique',
       'Un milieu de travail sûr et sain',
       "Obligation de prendre des mesures d'adaptation",
       'Obligation de prendre des mesures d\x92adaptation',
       'Problèmes de paye ou de rémunération',
       'Soutien afin de régler les problèmes de paye ou autre rémunération'],
      dtype=object)

In [9]:
data['INDICATORFRA'].unique()

array(['MILIEU DE TRAVAIL', 'EFFECTIF', 'MOBILISATION DES EMPLOYÉS',
       'LEADERSHIP', 'BIEN-ÊTRE EN MILIEU DE TRAVAIL', 'RÉMUNÉRATION'],
      dtype=object)

In [10]:
data[data['SCORE100'] == ' ']['QUESTION'].unique()

array(['Q01', 'Q02', 'Q03', 'Q04', 'Q05', 'Q06', 'Q07', 'Q08', 'Q09',
       'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18a',
       'Q18b', 'Q18c', 'Q18d', 'Q18e', 'Q18f', 'Q18g', 'Q18h', 'Q19',
       'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27', 'Q28',
       'Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36', 'Q37',
       'Q38', 'Q39', 'Q40', 'Q41', 'Q42', 'Q43a', 'Q43b', 'Q43c', 'Q43d',
       'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49', 'Q50', 'Q51', 'Q52',
       'Q53', 'Q54', 'Q55', 'Q56a', 'Q56b', 'Q56c', 'Q56d', 'Q56e',
       'Q56f', 'Q56g', 'Q57a', 'Q57b', 'Q57c', 'Q57d', 'Q57e', 'Q57f',
       'Q57g', 'Q57h', 'Q57i', 'Q57j', 'Q57k', 'Q57l', 'Q57m', 'Q58a',
       'Q58b', 'Q58c', 'Q58d', 'Q58e', 'Q58f', 'Q58g', 'Q58h', 'Q58i',
       'Q59a', 'Q59b', 'Q59c', 'Q59d', 'Q59e', 'Q59f', 'Q59g', 'Q59h',
       'Q59i', 'Q59j', 'Q59k', 'Q59l', 'Q59m', 'Q59n', 'Q59o', 'Q59p',
       'Q60', 'Q61', 'Q62', 'Q63a', 'Q63b', 'Q63c', 'Q63d', 'Q63e',
       'Q63f',

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1405579 entries, 0 to 1405578
Data columns (total 31 columns):
 #   Column                           Non-Null Count    Dtype 
---  ------                           --------------    ----- 
 0   LEVEL1ID                         1405579 non-null  int64 
 1   SURVEYR                          1405579 non-null  int64 
 2   BYCOND                           1405579 non-null  object
 3   DESCRIP_E                        1405579 non-null  object
 4   DESCRIP_F                        1405579 non-null  object
 5   DEMCODE                          1405579 non-null  int64 
 6   QUESTION                         1405579 non-null  object
 7   TITLE_E                          1405579 non-null  object
 8   TITLE_F                          1405579 non-null  object
 9   ANSWER1                          1405579 non-null  object
 10  ANSWER2                          1405579 non-null  object
 11  ANSWER3                          1405579 non-null  object
 12  

In [12]:
data.astype({'INDICATORID': 'int32', 'SUBINDICATORID': 'int32'}).dtypes

LEVEL1ID                            int64
SURVEYR                             int64
BYCOND                             object
DESCRIP_E                          object
DESCRIP_F                          object
DEMCODE                             int64
QUESTION                           object
TITLE_E                            object
TITLE_F                            object
ANSWER1                            object
ANSWER2                            object
ANSWER3                            object
ANSWER4                            object
ANSWER5                            object
ANSWER6                            object
ANSWER7                            object
MOST_POSITIVE_OR_LEAST_NEGATIVE    object
NEUTRAL_OR_MIDDLE_CATEGORY         object
MOST_NEGATIVE_OR_LEAST_POSITIVE    object
AGREE                              object
SCORE5                             object
SCORE100                           object
ANSCOUNT                           object
DEPT_E                            

In [13]:
SUBINDICATORID_to_str = {}
for i in data['SUBINDICATORID'].unique():
    strings = data[data['SUBINDICATORID']==i]['SUBINDICATORENG'].unique()
    print(strings[0])
    SUBINDICATORID_to_str[i] = strings[0]

Physical environment and equipment
Use of official languages
Job fit and development
Work-life balance and workload
Employee Engagement
Performance management
Organizational goals
Empowerment
Organizational performance
Ethical workplace
Diversity and inclusion
Anti-racism
Immediate supervisor
Senior management
Mobility and Retention
Harassment
Discrimination
Work-related stress
A psychologically healthy workplace
A safe and healthy workplace
Duty to accommodate
Pay or other compensation issues
Support to resolve pay or other compensation issues


In [14]:
SUBINDICATORID_to_str

{14: 'Physical environment and equipment',
 15: 'Use of official languages',
 5: 'Job fit and development',
 7: 'Work-life balance and workload',
 1: 'Employee Engagement',
 4: 'Performance management',
 9: 'Organizational goals',
 6: 'Empowerment',
 10: 'Organizational performance',
 13: 'Ethical workplace',
 11: 'Diversity and inclusion',
 12: 'Anti-racism',
 2: 'Immediate supervisor',
 3: 'Senior management',
 8: 'Mobility and Retention',
 16: 'Harassment',
 17: 'Discrimination',
 21: 'Work-related stress',
 20: 'A psychologically healthy workplace',
 19: 'A safe and healthy workplace',
 18: 'Duty to accommodate',
 22: 'Pay or other compensation issues',
 23: 'Support to resolve pay or other compensation issues'}

In [15]:
INDICATORID_to_str = {}
for i in data['INDICATORID'].unique():
    strings = data[data['INDICATORID']==i]['INDICATORENG'].unique()
    print(strings[0])
    INDICATORID_to_str[i] = strings[0]

WORKPLACE
WORKFORCE
EMPLOYEE ENGAGEMENT
LEADERSHIP
WORKPLACE WELL-BEING
COMPENSATION


In [16]:
INDICATORID_to_str

{4: 'WORKPLACE',
 3: 'WORKFORCE',
 1: 'EMPLOYEE ENGAGEMENT',
 2: 'LEADERSHIP',
 5: 'WORKPLACE WELL-BEING',
 6: 'COMPENSATION'}

In [17]:
np.save(data_path+'INDICATORID_to_str.npy', INDICATORID_to_str)
np.save(data_path+'SUBINDICATORID_to_str.npy', SUBINDICATORID_to_str)

In [34]:
data_Q_E = pd.read_excel(data_path+"2020-pses-supporting-documentation_document-de-reference-du-saff-2020.xlsx", sheet_name=2)
data_Q_R = pd.read_excel(data_path+"2020-pses-supporting-documentation_document-de-reference-du-saff-2020.xlsx", sheet_name=3)
data_DEM = pd.read_excel(data_path+"2020-pses-supporting-documentation_document-de-reference-du-saff-2020.xlsx", sheet_name=4, engine='openpyxl')
data_LVL = pd.read_excel(data_path+"2020-pses-supporting-documentation_document-de-reference-du-saff-2020.xlsx", sheet_name=5, engine='openpyxl')

# data_DEM = pd.read_csv(ExcelFile_data, 'DEMCODE', encoding='latin1')
# data_LVL = pd.read_csv(ExcelFile_data, 'DEMCODE', encoding='latin1')

In [41]:
data_Q_E.head()

,Question number/numéro de la question,English,Français
0,Q01,"Question 1. I have the tools, technology and e...","Question 1. Je dispose des outils, de la techn..."
1,Q02,Question 2. The material and tools provided fo...,Question 2. Le matériel et les outils mis à ma...
2,Q03,"Question 3. My physical environment (e.g., off...",Question 3. Mon environnement physique (p. ex....
3,Q04,Question 4. I get the training I need to do my...,Question 4. Je reçois la formation dont j'ai b...
4,Q05,"Question 5. I have the information, training a...","Question 5. J’ai l’information, la formation e..."


In [43]:
data_Q_E.columns

Index(['Question number/numéro de la question', 'English', 'Français'], dtype='object')

In [36]:
data_Q_R.head()

,QUESTION,Answer1ENG,Answer2ENG,Answer3ENG,Answer4ENG,Answer5ENG,Answer6ENG,Answer7ENG,Answer1FRA,Answer2FRA,Answer3FRA,Answer4FRA,Answer5FRA,Answer6FRA,Answer7FRA
0,Q01,Strongly agree,Somewhat agree,Neither agree nor disagree,Somewhat disagree,Strongly disagree,Don't know,Not applicable,Fortement d'accord,Plutôt d'accord,Ni d'accord ni en désaccord,Plutôt en désaccord,Fortement en désaccord,Ne sais pas,Ne s'applique pas
1,Q02,Strongly agree,Somewhat agree,Neither agree nor disagree,Somewhat disagree,Strongly disagree,Don't know,Not applicable,Fortement d'accord,Plutôt d'accord,Ni d'accord ni en désaccord,Plutôt en désaccord,Fortement en désaccord,Ne sais pas,Ne s'applique pas
2,Q03,Strongly agree,Somewhat agree,Neither agree nor disagree,Somewhat disagree,Strongly disagree,Don't know,Not applicable,Fortement d'accord,Plutôt d'accord,Ni d'accord ni en désaccord,Plutôt en désaccord,Fortement en désaccord,Ne sais pas,Ne s'applique pas
3,Q04,Strongly agree,Somewhat agree,Neither agree nor disagree,Somewhat disagree,Strongly disagree,Don't know,Not applicable,Fortement d'accord,Plutôt d'accord,Ni d'accord ni en désaccord,Plutôt en désaccord,Fortement en désaccord,Ne sais pas,Ne s'applique pas
4,Q05,Strongly agree,Somewhat agree,Neither agree nor disagree,Somewhat disagree,Strongly disagree,Don't know,Not applicable,Fortement d'accord,Plutôt d'accord,Ni d'accord ni en désaccord,Plutôt en désaccord,Fortement en désaccord,Ne sais pas,Ne s'applique pas


In [37]:
data_DEM.head()

,DEMCODE 2020,BYCOND 2020 (Demographic condition / condition démographique),DESCRIP_E,DESCRIP_F
0,1000,Q28 = 1,"You have requested flexible work hours, either...",Vous avez demandé un horaire de travail flexib...
1,1001,Q28 = 2,"You have requested flexible work hours, either...",Vous avez demandé un horaire de travail flexib...
2,1002,Q83 = 1,Supervisor,Superviseur(e)
3,1003,Q83 = 2,Non-supervisor,Pas superviseur(e)
4,1004,Q85 = 1,Managed employees who have been working remote...,À géré des employés qui travaillent à distance...


In [31]:
data_LVL.head()

,LEVEL1ID,LEVEL2ID,2020 Organisation Name (eng.),2020 Nom de l'organisation (fr.)
0,91,NaN,Administrative Tribunals Support Service of Ca...,Service canadien d’appui aux tribunaux adminis...
1,9,NaN,Agriculture and Agri-Food Canada,Agriculture et Agroalimentaire Canada
2,32,NaN,Atlantic Canada Opportunities Agency,Agence de promotion économique du Canada atlan...
3,83,NaN,Canada Border Services Agency,Agence des services frontaliers du Canada
4,34,NaN,Canada Economic Development for Quebec Regions,Développement économique Canada pour les régio...


In [51]:
data_DEM.columns

Index(['DEMCODE 2020',
       'BYCOND 2020 (Demographic condition / condition démographique)',
       'DESCRIP_E', 'DESCRIP_F'],
      dtype='object')

In [72]:
i = 101
QuestionID_to_strEnglish = {}
for Question in data_Q_E['English'].unique():
    QuestionID_to_strEnglish[i] = Question
    i += 1
QuestionID_to_strEnglish

{101: 'Question 1. I have the tools, technology and equipment I need to do my job.',
 102: 'Question 2. The material and tools provided for my work, including software and other automated tools, are available in the official language of my choice.',
 103: 'Question 3. My physical environment (e.g., office, workspace) is suitable for my job requirements.',
 104: 'Question 4. I get the training I need to do my job.',
 105: 'Question 5. I have the information, training and equipment I need to ensure my health and safety at work.',
 106: 'Question 6. I have support at work to balance my work and personal life.',
 107: 'Question 7. I get a sense of satisfaction from my work.',
 108: 'Question 8. I receive meaningful recognition for work well done.',
 109: 'Question 9. I have clear work objectives.',
 110: "Question 10. I know how my work contributes to the achievement of my department's or agency's goals.",
 111: 'Question 11. Overall, I feel valued at work.',
 112: 'Question 12. I am proud

In [56]:
i = 2011
j = 2069

DEMCODE_to_strEnglish = {}
for info in data_DEM[(data_DEM['DEMCODE 2020'] >= i) & (data_DEM['DEMCODE 2020'] <= j)]['DESCRIP_E']:
    DEMCODE_to_strEnglish[i] = info
    i += 1
DEMCODE_to_strEnglish

{2011: 'Male gender',
 2012: 'Female gender',
 2013: 'Gender diverse',
 2014: 'Indigenous ',
 2015: 'Non-Indigenous ',
 2016: 'First Nation (North American Indian)',
 2017: 'Métis',
 2018: 'Inuk (Inuit)',
 2019: 'Person with a disability',
 2020: 'Not a person with a disability',
 2021: 'A seeing disability affects vision, including total blindness, partial sight and visual distortion',
 2022: 'A seeing disability affects vision, including total blindness, partial sight and visual distortion - Not selected',
 2023: 'A hearing disability affects ability to hear, including being hard of hearing, deafness or acoustic distortion',
 2024: 'A hearing disability affects ability to hear, including being hard of hearing, deafness or acoustic distortion - Not selected',
 2025: 'A mobility issue affects ability to move your body, including the required use of a wheelchair or a cane, or other issues impacting your mobility',
 2026: 'A mobility issue affects ability to move your body, including the

In [58]:
data_LVL.columns

Index(['LEVEL1ID', 'LEVEL2ID', '2020 Organisation Name (eng.)',
       '2020 Nom de l'organisation (fr.)'],
      dtype='object')

In [59]:
data_LVL['LEVEL2ID'].unique()

array([ nan, 121., 103., 144., 114., 105., 109., 149., 115., 107., 119.,
       101., 104., 110., 139., 147., 151., 143., 111., 145., 125., 150.])

In [70]:

LVLID_to_strEnglish = {}
for index, line in data_LVL[data_LVL['LEVEL2ID'].isnull()].iterrows():
    code, descr = line['LEVEL1ID'], line['2020 Organisation Name (eng.)']
    LVLID_to_strEnglish[code] = descr

LVLID_to_strEnglish

{91: 'Administrative Tribunals Support Service of Canada',
 9: 'Agriculture and Agri-Food Canada',
 32: 'Atlantic Canada Opportunities Agency',
 83: 'Canada Border Services Agency',
 34: 'Canada Economic Development for Quebec Regions ',
 75: 'Canada Energy Regulator ',
 1: 'Canada Revenue Agency',
 78: 'Canada School of Public Service',
 86: 'Canadian Food Inspection Agency',
 25: 'Canadian Grain Commission',
 20: 'Canadian Heritage',
 41: 'Canadian Human Rights Commission',
 35: 'Canadian Institutes of Health Research',
 69: 'Canadian Nuclear Safety Commission',
 31: 'Canadian Radio-television and TeleCommunications Commission ',
 51: 'Canadian Security Intelligence Service',
 33: 'Canadian Space Agency',
 54: 'Canadian Transportation Agency',
 89: 'Communications Security Establishment Canada',
 4: 'Correctional Service Canada',
 30: 'Courts Administration Service',
 22: 'Crown-Indigenous Relations and Northern Affairs Canada',
 24: 'Department of Finance Canada',
 19: 'Department o

In [ ]:
np.save(data_path+'QuestionID_to_strEnglish.npy', QuestionID_to_strEnglish)
np.save(data_path+'DEMCODE_to_strEnglish.npy', DEMCODE_to_strEnglish)
np.save(data_path+'LVLID_to_strEnglish.npy', LVLID_to_strEnglish)